In [ ]:
# Author : Chhitij.s

In [6]:
import pandas as pd
import scipy.stats
import math;
from enum import Enum # for making option as enum

In [23]:
class Option(Enum):
      INTERVAL_ESTIMATION, POINT_ESTIMATION ,HYPOTHESIS , ALL= range(4) # 0,1,2,3

# Defined zAlfa value default for onetail and two tails testing
oneTailZalfa = 1.64;
twoTailZalfa = 1.96;

tAlfaValue = {'DOF15': {'T.005':1.753, 'T.025':2.131},'DOF16': {'T.005':1.746, 'T.025':2.120}}

        
# Function for handling exception for entered file val
def handleException(dataset):
        try:
            # from cereal data set extract panda series column sodim
            dataset = pd.read_csv(dataset, usecols=['sodium'], squeeze=True);
        except IOError:
            print("Error: can\'t find file")
        else:
            return dataset


#@arg fileType: for csv file,
#@arg flag: for calculation oneTail or twoTail test default is two tail test
#@arg populationMean: for calculation hypothisys;
#@arg option : for caluation of Option enum class. Default option ALL

def hyp_test(fileType,flag="TwoTail",option=Option.ALL, populationMean=0):
    estimate = 0;
    output = {}
    try:
        # from cereal data set extract panda series column sodim
        df = pd.read_csv(fileType, usecols=['sodium'], squeeze=True);
        mean = df.mean();
        sd = df.std();
        sampleSize = df.count();
    except IOError:
            print("Error: can\'t find file")
    else:
        # check if in option hypothesis or all is passed
        if(option==Option.HYPOTHESIS):
                decesion = hTesting(flag,mean,sd,sampleSize,populationMean);
                decesion_estimation = {'Decision Of Hypothysis':decesion}
                output.update(decesion_estimation);
        elif(option==Option.INTERVAL_ESTIMATION):
            # if sample size lessthan 30 then have to calculate for t test
            if(sampleSize <=30):
                estimate = tEstimation(mean,sd,sampleSize);
                interval_estimation = {'Interval Estimate':estimate}
                output.update(interval_estimation);
            # sample size morethan 30 is for z esrimation;
            else:
                estimate = zEstimation(flag,mean,sd,sampleSize);
                interval_estimation = {'Interval Estimate':estimate}
                output.update(interval_estimation);
            # case for point estimeate
        elif(option==Option.POINT_ESTIMATION):
            estimate = "{:.2f}".format(mean);
            point_estimate = {'Point Estimate': estimate}
            output.update(point_estimate)
        else:
            # Update Point Estimate
            estimate = "{:.2f}".format(mean);
            point_estimate = {'Point Estimate': estimate}
            output.update(point_estimate)
            # Update Interval Estimate .
            if(sampleSize <=30):
                estimate = tEstimation(mean,sd,sampleSize);
                interval_estimation = {'Interval Estimate':estimate}
                output.update(interval_estimation);
            # sample size morethan 30 is for z esrimation;
            else:
                estimate = zEstimation(flag, mean,sd,sampleSize);
                interval_estimation = {'Interval Estimate':estimate}
                output.update(interval_estimation);
            # Update Hypothysis Decision .
            decesion = hTesting(flag,mean,sd,sampleSize,populationMean);
            decesion_estimation = {'Decision Of Hypothysis':decesion}
            output.update(decesion_estimation);
    
    return output;


# consedering aTlfa value is oneTail test for now
# Need to optmise more for t table and z table
# @param mean : mean for sample
# @param sd : standerd davitaion of sample
# @param samplesize: size of the sample
def tEstimation(mean,sd,sampleSize):
            # Assuming degree of freedom is 15  sample size = 16-1 = 15;
            plusMarginValue = (mean+(tAlfaValue['DOF15']['T.005'])*(sd/math.sqrt(sampleSize)));
            minusMarginValue = (mean-(tAlfaValue['DOF15']['T.005'])*(sd/math.sqrt(sampleSize)));
            return ("{:.2f}".format(plusMarginValue),"{:.2f}".format(minusMarginValue));


# consedering zAlfaValue is twotail test for now
# @param mean : mean for sample
# @param sd : standerd davitaion of sample
# @param samplesize: size of the sample
def zEstimation(flag,mean,sd,sampleSize):
            print('mean',mean)
            if(flag=='TwoTail'):
                plusMarginValue = (mean+(twoTailZalfa)*(sd/math.sqrt(sampleSize)));
                minusMarginValue = (mean-(twoTailZalfa)*(sd/math.sqrt(sampleSize)));
            else:
                plusMarginValue = (mean+(oneTailZalfa)*(sd/math.sqrt(sampleSize)));
                minusMarginValue = (mean-(oneTailZalfa)*(sd/math.sqrt(sampleSize)));
            return ("{:.2f}".format(plusMarginValue),"{:.2f}".format(minusMarginValue));

# param flag for one tail testing or two tail testing
# @param mean : mean for sample
# @param sd : standerd davitaion of sample
# @param samplesize: size of the sample
# @param populationMean: respect to perform hypothsys testing 
def hTesting(flag,mean,sd,sampleSize,populationMean):
            desicion = '';
            xBarMinusMu = populationMean - mean;
            sdAndRootOfSample =  sd/math.sqrt(sampleSize);
            zStatesScore = xBarMinusMu /sdAndRootOfSample;
            if(flag=='TwoTail'):
                desicion = 'Reject Hypothysis' if(zStatesScore > twoTailZalfa) else 'Accept Hypothsis';
                
            return desicion;

In [33]:
# make arg to call hyp_test function
#arg 1 : file
#arg2: flag for twotail testing or one tail testing
#arg3: for option
#arg4: for external mean in case of all and hypothys testing

# IMP INPUT POINT ----------------------------------------------------
# For INTERVAL_ESTIMATION provide Option.INTERVAL_ESTIMATION enum
# For POINT_ESTIMATION porvide Option.POINT_ESTIMATION from option enum
# For HYPOTHESIS porvide Option.HYPOTHESIS 
# For All provide Option.ALL
# default value of Option is seted Option.ALL

# As its function for one sample test then for Performing hypotysis please provide performed population means 
xBarMean = 221;
args = ("cereal.csv",'TwoTail', Option.ALL , xBarMean)
hyp_test(*args)

mean 159.67532467532467


{'Point Estimate': '159.68',
 'Interval Estimate': ('178.40', '140.95'),
 'Decision Of Hypothysis': 'Reject Hypothysis'}